# Preview dataset

In [ ]:
import os

def fetch_images_from_path(path):
    lst = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if '.jpg' in file:
                lst.append(root + file)
    return lst

In [ ]:
import os

dir_path = '../data/kaggle_dogcat'

train_dogs_path = dir_path + '/train/dogs/'
train_cats_path = dir_path + '/train/cats/'
test_images_path  = dir_path + '/test/'

train_dogs = fetch_images_from_path(train_dogs_path)
train_cats = fetch_images_from_path(train_cats_path)
test_images  = fetch_images_from_path(test_images_path)

In [ ]:
print(train_dogs[:10])

In [ ]:
num_train_dogs = len(train_dogs)
num_train_cats = len(train_cats)
num_test_images = len(test_images)

In [ ]:
print(num_train_dogs)

# Preprocessing

## Resize image

In [ ]:
processed_dir = '../data/kaggle_dogcat/processed'

image_size = (224,224)

In [ ]:
from PIL import Image

def resize_image_to_target_path(source_list ,store_path, image_size):
    cnt = 0
    for file in source_list:
        img = Image.open(file)
        resized_img = img.resize(image_size)
        resized_img.save(store_path + '{0}.jpg'.format(cnt))
        cnt += 1

In [ ]:
resize_image_to_target_path(train_dogs, processed_dir + '/train/dogs/', image_size)
resize_image_to_target_path(train_cats, processed_dir + '/train/cats/', image_size)
resize_image_to_target_path(test_images, processed_dir + '/test/', image_size)

## Combine dataset

In [ ]:
processed_train_dogs = fetch_images_from_path(processed_dir + '/train/dogs/')
processed_train_cats = fetch_images_from_path(processed_dir + '/train/cats/')

In [ ]:
def copy_image_to_target_path(dog_files, cat_files, path):
    cnt = 0
    for file in dog_files:
        img = Image.open(file)
        img.save(path + '/{0}.jpg'.format(cnt))
        cnt += 1
    for file in cat_files:
        img = Image.open(file)
        img.save(path + '/{0}.jpg'.format(cnt))
        cnt += 1

In [ ]:
copy_image_to_target_path(processed_train_dogs, processed_train_cats, processed_dir + '/train')

# Load dataset

In [22]:
processed_dir = '../data/kaggle_dogcat/processed'

image_size = (224,224)

In [19]:
from PIL import Image
import numpy as np

def open_images_as_numpy_array(path, nums):
    lst = []
    for i in range(nums):
        fname = path + '{0}.jpg'.format(i)
        img = Image.open(fname)
        lst.append(np.array(img))
    return np.array(lst)

In [ ]:
x_train = open_images_as_numpy_array(processed_dir + '/train/', 4000)
y_train = [0]*2000 + [1]*2000

In [ ]:
x_train.shape

## Training

In [ ]:
import keras

x_train = x_train.astype('float32') / 255
y_train = keras.utils.to_categorical(y_train, num_classes=2, dtype='float32')

In [ ]:
from random import shuffle

lst = []

for image, label in zip(x_train, y_train):
    lst.append((image, label))

shuffle(lst)

In [ ]:
for i in range(10):
    image, label = lst[i]
    print(label)

In [ ]:
x_train_lst = []
y_train_lst = []

for pair in lst:
    image, label = pair
    x_train_lst.append(image)
    y_train_lst.append(label)
    
x_train = np.array(x_train_lst)
y_train = np.array(y_train_lst)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
print(y_train[:10])

In [1]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
DATASET_PATH  = '../data/kaggle_dogcat/processed'
IMAGE_SIZE    = (224, 224)
BATCH_SIZE    = 32
NUM_CLASSES   = 2
FREEZE_LAYERS = 2
NUM_EPOCHS    = 50

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'


train_datagen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   channel_shift_range=5,
                                   horizontal_flip=True,
                                   rescale=1/.255,
                                   fill_mode='nearest')

train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator(rescale=1/.255)
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Using TensorFlow backend.


Found 3200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Class #0 = cats
Class #1 = dogs


In [8]:
from tensorflow.python.keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import keras
import numpy as np
from sklearn.metrics import roc_auc_score    

'''
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)
x = Dropout(0.2)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
'''
from keras.models import Sequential

vgg16_model = keras.applications.vgg16.VGG16(weights='vgg16_weights_tf_dim_ordering_tf_kernels.h5')

model = Sequential()

for layer in vgg16_model.layers:
    layer.trainable = False

for layer in vgg16_model.layers[:-1]:
    model.add(layer)
    
model.add(Dense(2, activation='softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [10]:
# 設定凍結與要進行訓練的網路層
'''
for layer in net_final.layers:
    layer.trainable = False

net_final.layers[-1].trainable = True
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
'''
from keras.optimizers import Adam

model.compile(Adam(lr=.00002122), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# 儲存訓練好的模型
model.save(WEIGHTS_FINAL)

W0818 01:23:26.703009  7656 deprecation.py:323] From C:\Users\Scherzando\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
100/100 [==============================] - 44s 438ms/step - loss: 2.4850 - acc: 0.5962 - val_loss: 1.4533 - val_acc: 0.7075
Epoch 2/50
100/100 [==============================] - 41s 407ms/step - loss: 1.1315 - acc: 0.7613 - val_loss: 0.8904 - val_acc: 0.8237
Epoch 3/50
100/100 [==============================] - 39s 389ms/step - loss: 0.7683 - acc: 0.8238 - val_loss: 0.7215 - val_acc: 0.8550
Epoch 4/50
100/100 [==============================] - 38s 379ms/step - loss: 0.6436 - acc: 0.8491 - val_loss: 0.6098 - val_acc: 0.8725
Epoch 5/50
100/100 [==============================] - 38s 379ms/step - loss: 0.5517 - acc: 0.8650 - val_loss: 0.5478 - val_acc: 0.8875
Epoch 6/50
100/100 [==============================] - 38s 377ms/step - loss: 0.5122 - acc: 0.8734 - val_loss: 0.4941 - val_acc: 0.9038
Epoch 7/50
100/100 [==============================] - 38s 378ms/step - loss: 0.4374 - acc: 0.8869 - val_loss: 0.4600 - val_acc: 0.9075
Epoch 8/50
100/100 [==============================] - 3

In [29]:
print(DATASET_PATH + '/test')

test_datagen = ImageDataGenerator(rescale=1/.255)
test_batches = test_datagen.flow_from_directory(DATASET_PATH + '/test',
                                                target_size=IMAGE_SIZE,
                                                class_mode='categorical',
                                                shuffle=False,
                                                batch_size=1)

../data/kaggle_dogcat/processed/test
Found 400 images belonging to 1 classes.


In [35]:
x_test = open_images_as_numpy_array(processed_dir + '/test/test/', 400)

In [36]:
x_test = x_test.astype('float32') / 255

In [39]:
y_test = model.predict(x_test)

In [ ]:
net_final.load_weights(WEIGHTS_FINAL)

In [33]:
y_test = model.predict_generator(test_batches, steps=400)

In [40]:
y_test[:10]

array([[0.27359673, 0.72640324],
       [0.22123948, 0.77876055],
       [0.37720594, 0.62279403],
       [0.2955681 , 0.70443195],
       [0.29024053, 0.7097595 ],
       [0.36384672, 0.6361533 ],
       [0.26339388, 0.7366062 ],
       [0.32249004, 0.67750996],
       [0.31364888, 0.6863511 ],
       [0.3923276 , 0.60767233]], dtype=float32)

In [41]:
import csv

with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID','Predicted'])
    col_id = 0
    for col in y_test:
        val = col[0]
        if val < 0.1:
            val = 0.0
        writer.writerow([str(col_id).zfill(3), val])
        col_id += 1

# Result

<img src="final.png">